# Register Best Model in MLFlow Model Registry

In [1]:
%load_ext autoreload
%autoreload 2

::: {.content-hidden}
Import necessary Python modules
:::

In [2]:
import json
import os
import sys
from glob import glob

import mlflow.sklearn
import pandas as pd
from mlflow import MlflowClient

::: {.content-hidden}
Get relative path to project root directory
:::

In [3]:
PROJ_ROOT_DIR = os.path.join(os.pardir)
src_dir = os.path.join(PROJ_ROOT_DIR, "src")
sys.path.append(src_dir)

::: {.content-hidden}
Import custom Python modules
:::

In [4]:
%aimport model_helpers
import model_helpers as modh

## About

This step retrieves the best ML model across all the MLFlow experiment runs that were tracked during ML development. This best model is then registered in the MLFlow Model Registry.

## User Inputs

Define the primary ML scoring metric

In [5]:
#| echo: true
primary_metric = "fbeta2"

::: {.content-hidden}
Get path to data sub-folders
:::

In [6]:
data_dir = os.path.join(PROJ_ROOT_DIR, "data")
raw_data_dir = os.path.join(data_dir, "raw")

::: {.content-hidden}
Define MLFlow storage paths
:::

In [7]:
mlruns_db_fpath = f"{raw_data_dir}/mlruns.db"
mlflow.set_tracking_uri(f"sqlite:///{mlruns_db_fpath}")

::: {.content-hidden}
Set environment variable to silence MLFlow `git` warning messsage
:::

In [8]:
os.environ["GIT_PYTHON_REFRESH"] = "quiet"

## Manage ML Experiments

### Inspect Experiment Run Outputs

Get all runs of all experiments

In [ ]:
#| echo: true
df_expt_runs = modh.get_all_experiment_runs()

In [9]:
#| output: false
with pd.option_context("display.max_columns", None):
    display(df_expt_runs.drop(columns=["params", "column_names"]))

,resampling_approach,clf,param_clf__a,param_clf__b,param_preprocessor__cat__rarecats__fe__ignore_format,param_preprocessor__cat__rarecats__fe__n_categories,param_preprocessor__cat__rarecats__fe__replace_with,param_preprocessor__cat__rarecats__fe__tol,param_resampler__sampling_strategy,param_select__threshold,test_accuracy,test_balanced_accuracy,test_precision,test_recall,test_roc_auc,test_f1,test_fbeta05,test_fbeta2,test_pr_auc,test_avg_precision,fit_time,score_time,experiment_run_type,train_val_accuracy,train_val_balanced_accuracy,train_val_precision,train_val_recall,train_val_roc_auc,train_val_f1,train_val_fbeta05,train_val_fbeta2,train_val_pr_auc,train_val_avg_precision,train_start_date,test_end_date,num_observations,num_columns,experiment_id,run_id
0,os,BetaDistClassifier,0.2,2.31,True,1,other,0.1,0.1,0.7,0.937904,0.501539,0.501726,0.501539,0.501539,0.501568,0.501646,0.501536,0.034591,0.034205,0.836648,0.118831,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
1,os,BetaDistClassifier,0.2,2.25,True,1,other,0.1,0.1,0.7,0.935543,0.496308,0.496082,0.496308,0.496308,0.496183,0.496119,0.496255,0.032233,0.033905,0.81209,0.115707,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
2,os,BetaDistClassifier,0.2,2.35,True,1,other,0.1,0.1,0.7,0.939179,0.502867,0.503343,0.502867,0.502867,0.502983,0.503166,0.502888,0.035375,0.034318,0.802264,0.122692,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
3,os,BetaDistClassifier,0.2,2.4,True,1,other,0.1,0.1,0.7,0.940171,0.501376,0.501676,0.501376,0.501376,0.501346,0.501493,0.501327,0.035538,0.034193,0.80081,0.117762,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
4,os,BetaDistClassifier,0.2,2.5,True,1,other,0.1,0.1,0.7,0.94206,0.496341,0.495071,0.496341,0.496341,0.495448,0.495144,0.495935,0.034335,0.033921,0.789521,0.116408,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
5,os,BetaDistClassifier,0.3,2.31,True,1,other,0.1,0.1,0.7,0.919866,0.498214,0.498741,0.498214,0.498214,0.497814,0.498251,0.497846,0.034406,0.033984,0.829881,0.126483,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
6,os,BetaDistClassifier,0.3,2.25,True,1,other,0.1,0.1,0.7,0.91741,0.498947,0.499295,0.498947,0.498947,0.498194,0.498695,0.498343,0.035797,0.034027,0.815624,0.116943,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
7,os,BetaDistClassifier,0.3,2.35,True,1,other,0.1,0.1,0.7,0.921188,0.497562,0.498231,0.497562,0.497562,0.497366,0.497787,0.497321,0.033549,0.033949,0.822759,0.117708,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
8,os,BetaDistClassifier,0.3,2.4,True,1,other,0.1,0.1,0.7,0.923785,0.49757,0.498133,0.49757,0.49757,0.497508,0.497816,0.497444,0.03559,0.03395,0.800312,0.12295,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398
9,os,BetaDistClassifier,0.3,2.5,True,1,other,0.1,0.1,0.7,0.928413,0.505978,0.504998,0.505978,0.505978,0.505274,0.505064,0.505637,0.032603,0.034598,0.79485,0.117124,nested,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,20160901,20170228,113728,28,1,742cf152d5094be0b686edf910c7d398


### Get Outputs of Best Experiment Run

In [ ]:
#| echo: true
df_best_expt_run = modh.get_best_experiment_run(
    df_expt_runs, "experiment_run_type == 'parent'", f"test_{primary_metric}"
)

In [10]:
#| output: false
with pd.option_context("display.max_columns", None):
    display(df_best_expt_run.to_frame().transpose())

,resampling_approach,clf,param_clf__a,param_clf__b,param_preprocessor__cat__rarecats__fe__ignore_format,param_preprocessor__cat__rarecats__fe__n_categories,param_preprocessor__cat__rarecats__fe__replace_with,param_preprocessor__cat__rarecats__fe__tol,param_resampler__sampling_strategy,param_select__threshold,params,test_accuracy,test_balanced_accuracy,test_precision,test_recall,test_roc_auc,test_f1,test_fbeta05,test_fbeta2,test_pr_auc,test_avg_precision,fit_time,score_time,experiment_run_type,train_val_accuracy,train_val_balanced_accuracy,train_val_precision,train_val_recall,train_val_roc_auc,train_val_f1,train_val_fbeta05,train_val_fbeta2,train_val_pr_auc,train_val_avg_precision,train_start_date,test_end_date,num_observations,num_columns,column_names,experiment_id,run_id
30,os,BetaDistClassifier,0.3,2.5,True,1,other,0.1,0.1,0.7,"{""clf__a"": 0.3, ""clf__b"": 2.5, ""preprocessor__...",0.940538,0.499218,0.022135,0.499218,0.499218,0.498453,0.498946,0.498594,0.023227,0.023027,<NA>,<NA>,parent,0.919483,0.501968,0.047763,0.501968,0.501968,0.502007,0.50206,0.501977,0.043754,0.043898,20160901,20170228,113728,28,"[""fullvisitorid"", ""visitId"", ""visitNumber"", ""v...",1,742cf152d5094be0b686edf910c7d398


### Get Parameters Associated With Best Experiment Run

Get the metadata and metrics for all available data and experiment run ID for best performing run

1. features
   - list of column names
2. metrics
   - primary metric score on the test split, during ML evaluation
3. run ID
   - MLFlow experiment run ID

In [ ]:
#| echo: true
cols_best_expt_run = json.loads(df_best_expt_run["column_names"])
best_model_eval_score = df_best_expt_run[f"test_{primary_metric}"]
best_run_id = df_best_expt_run["run_id"]

In [11]:
#| output: false
print(best_model_eval_score)
cols_best_expt_run

0.4985942


['fullvisitorid',
 'visitId',
 'visitNumber',
 'visitStartTime',
 'quarter',
 'month',
 'day_of_month',
 'day_of_week',
 'hour',
 'minute',
 'second',
 'source',
 'medium',
 'channelGrouping',
 'hits',
 'bounces',
 'last_action',
 'promos_displayed',
 'promos_clicked',
 'product_views',
 'product_clicks',
 'pageviews',
 'time_on_site',
 'browser',
 'os',
 'deviceCategory',
 'added_to_cart',
 'made_purchase_on_future_visit']

### Get Name of Logged Model Associated with Best Experiment Run

Get name of model associated with best run

In [ ]:
#| echo: true
df_best_run_model = modh.get_single_registered_model(f"run_id == '{best_run_id}'")
best_run_model_name = df_best_run_model.squeeze()["name"]

In [12]:
# output: false
with pd.option_context("display.max_colwidth", None):
    display(df_best_run_model)
print(best_run_model_name)

,name,run_id,description,source,version,status
0,BetaDistClassifier_20160901_20170228_133892_feats__20230605_225228,742cf152d5094be0b686edf910c7d398,Best BetaDistClassifier model with fbeta2 score of 0.4985941872,/home/jovyan/notebooks/mlruns/742cf152d5094be0b686edf910c7d398/artifacts/model,1,READY


BetaDistClassifier_20160901_20170228_133892_feats__20230605_225228


### Load Best Deployment Candidate Model from Model Registry

In [ ]:
best_model_uri = f"models:/{best_run_model_name}/latest"
model = mlflow.sklearn.load_model(model_uri=best_model_uri)

In [13]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('aboveavg',
                                                                   AboveAveragePagePromoEngager(cols=['hits',
                                                                                                      'promos_displayed',
                                                                                                      'promos_clicked',
                                                                                                      'product_views',
                                                                                                      'product_clicks',
                                                                                                      'pageviews',
                                                                                                      'time_on_site'])),
                                                                  ('scaler',
                                                                   MinMaxScaler())]),
                                                  ['hits', 'promos_displayed',
                                                   'promos_clicked',
                                                   'product_views',
                                                   'product_clicks',
                                                   'page...
                                                                  ('dummy',
                                                                   OneHotEncoder(drop='first',
                                                                                 dtype=<class 'int'>,
                                                                                 handle_unknown='ignore'))]),
                                                  ['bounces', 'last_action',
                                                   'source', 'medium',
                                                   'channelGrouping', 'browser',
                                                   'os', 'deviceCategory'])])),
                ('select', DropCorrelatedFeatures(threshold=0.7)),
                ('resampler',
                 RandomOverSampler(random_state=88, sampling_strategy=0.1)),
                ['clf', BetaDistClassifier(a=0.3, b=2.5, random_state=88)]])

### Add MLFlow Model Associated with Best Experiment Run to Model Registry

Create a new registered model, with a version

In [14]:
#| echo: true
client = MlflowClient(tracking_uri=mlflow.get_tracking_uri())
result = client.create_model_version(
    name=best_run_model_name,
    await_creation_for=None,
    tags={'deployment-candidate': "yes"},
    description=(
        f"Best Model based on {primary_metric} score of "
        f"{best_model_eval_score:.10f}"
    ),
    source=f"mlruns/{best_run_id}/artifacts/model",
    run_id=best_run_id,
)

## Next Step

The registered model will be used during downstream steps such as inference to make predictions for first-time visitors to the store during the production period (following the end of the test data split).